# Predictive Analytics Exam 1 - Evan Callaghan

### Topics: 

### 1. True or False

#### a) In class imbalanced classification tasks, the goal is to find the model that has the highest accuracy. FALSE

#### b) Over-fitting is the biggest issue with over-sampling. TRUE

#### c) Under-fitting is the biggest issue with over-sampling. FALSE

#### d) A model with a large number of input variables always have a good performance. FALSE

#### e) If we run the SMOTE sampling technique multiple times on an imbalanced dataset, we will always obtain the same synthetic dataset. FALSE

#### f) Having access to good data is better than having a good model. TRUE

#### g) RFE works with any support vector machine model. FALSE

#### h) RFE doesn’t work with any support vector machine model. FALSE (works with linear kernel)

#### i) A typical approach to engineer features is to consider interactions because most interactions help models to generate good predictions. TRUE

#### j) The support vectors in support vector machine models can be used to identify important variables/featues. TRUE

### 2. How does the Recursive Feature Elimination (RFE) algorithm work? Be specific.

#### The RFE algorithm is essentially a backwards variable selection process. First, a model is built with all predictor variables and the corresponding variable importane scores are computed. Then, the least important predictor variable is dropped, another model is built, and the variable importance scores are computed again. The specific variable subset that produces the optimal model is the one that it used to build the final model.

### 3. Explain one-hot encoding. Be specific.

#### One-hot encoding is the process of converting catagorical data into a form that can be easily used by a machine learning algorithm to produce better predictions. In this process, all categorical values for a particular variable are created into a categorical column and assigned a value of zero or one. For example: if the data set has a variable "Major" with the values Math, CS, and Business Analytics, one-hot encoding creates three new variables to represent each particular value with a binary label. 

### 4. If you have a date column in your data-frame, then how will you perform feature engineering on the date column? List at least three features that you will engineer from date. Be specific.

#### Depending on the context of the problem, three possible variables to engineer are: day of the week, day of the month, or month. Day of the week could be important when looking into fraud transactions (they may be more common on weekends). Day of the month could be important when looking at credit card defaults (what day of the month do people typically pay thier bills? Is it close to the limit?). And month could be important when modeling revenue for a store (cyclical trend and some months are not as popular). 

### 5. In what scenarios, would you prefer to use the precision-recall curve instead of ROC curve to measure the performance of a classifier? Be specific.

#### 

### 6. Explain the biggest drawback of one-vs-one multi-class classification when compared to one-vs-all multi-class classification. Be specific.

#### 

*** ### 7. A data scientist is building a linear regression model. One of the input variables is a categorical variable with three labels. So, he decided to use the one-hot encoding approach to transform the categorical variable into dummy variables. How many dummy variables does he need to include in the linear model?

#### D) 3

### 8. Suppose you are building a fraud detection system for major US bank. You have access to all the transaction data for the past week for users (date, location, and amount). What kind of new features can you engineer? Be creative and list at least three features that you would engineer for the fraud detection system.

### When looking at the problem of fraud detection, I would engineer the following variables: Distance - the distance between the purchase location and registered home address, Hour of the Day - the hour of the transaction, and Time Since Last Transaction - the time in minutes between two consecutive transactions for a particular customer. 

### 9. Let’s assume that you are the data scientist in charge of this project. The goal is to build a classification system that can flag future credit card payments that are likely to default. Answer the following: a) Is the dataset imbalanced? Be specific. b)  Using the above chart, engineer one feature for your classification model. Be specific.

#### a) The data set is certainly imbalanced. As an estimation, there are probably seven or eight times as many zero class labels compared to one class labels. 

#### b) 

### 10. Which of the following is/are TRUE about feature subset selection?

#### 

### 11. A data scientist is building a regression model. A few of the input variables are categorical, and he has not looked at the distribution of the categorical data in the test data. The data scientist wants to apply the one-hot encoding on the categorical features. What challenges he may face if he have applied one-hot encoding on a categorical variable of the train dataset?

#### 

### 12. What is the difference between feature engineering and feature selection? Be specific.

#### Feature engineering is the process of transforming or combining existing data into new features that may help uncover relationships in the data, and ultimately, help the model to make better predictions. On the other hand, feature selection is the process of carefully selecting which input variables (original or engineered) should be included in the final model. This is important because having a manageable number of inputs reduces computing time and including unimportant variables in the model often hurts the prediction power.  

### 13. Based on the discussions from Chapter 4, list at least two benefits of feature selection. Be specific.

Two benefits:

### 14. 